In [3]:
import plotly.express as px
import plotly.graph_objects as go 

import ipywidgets as wg
from IPython.display import display, HTML

import pandas as pd
import numpy as np

import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')
import scipy


# Simple example of using both

In [ ]:
x = np.linspace(0, np.pi, 1000)

layout = go.Layout(
    title = "SIMPLE EXAMPLE",           # fig.layout.title = "SIMPLE EXAMPLE"
    yaxis = {"title": "volts"},         # fig.layout.yaxis.title = "volts"
    xaxis = {"title": "nanoseconds"},
    width = 500,
    height = 300,
    margin = dict(l=40,r=40,t=60,b=40)
)

@wg.interact
def update_plot(signals = wg.SelectMultiple(options=list(range(6)), value=(0, ), description = "Bessel Order"), 
                freq = wg.FloatSlider(min=1, max=20, value=1, desription="Freq")):
    
    fig = go.Figure(layout = layout)
    
    for s in signals:
        
        trace = go.Scatter(
            x = x,
            y = scipy.special.jv(s, freq * x), 
            mode = "lines",
            name = f"Bessel {s}", 
        )
        
        fig.add_traces(trace)

    fig.show()


In [5]:
#This introduces the go.FigureWidget, object which has identical syntax 
# to the Plotly go.Figure, but it is also treated like a 
# widget in the ipywidgets library 
# (i.e. you don't need to create an output widget to 
# put the graph in; the graph itself is an output widget).

#This also uses the fig.batch_update() method. 
# This is a good way to update FigureWidgets in your functions. 
# It sends all the updates at the same time, rather than 
# one at a time (which can create flickering).

#P.S. — This example is probably the most 
# complicated in this whole notebook, so don't worry 
# if it seems hard to interpret at first.

df = pd.read_csv('https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)

month_widget = wg.IntSlider(min=1, max=12, step=1, value=1, description='Month:', continuous_update=False)
use_date_widget = wg.Checkbox(value=True, description='Date: ')
airline_widget = wg.Dropdown(description='Airline:   ', options=df['carrier'].unique(), value='DL')
origin_widget = wg.Dropdown(description='Origin Airport:', options=df['origin'].unique(), value='LGA')

trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
data2 = [trace1, trace2]

layout2 = go.Layout(title='NYC FlightDatabase', 
                    barmode='overlay', 
                    xaxis={"title":"Delay in Minutes"}, 
                    yaxis={"title":"Number of Delays"})

fig_hist = go.FigureWidget(data=data2, layout=layout2)


def update_histogram(change):

    filter_list = (df["carrier"] == airline_widget.value) & (df["origin"] == origin_widget.value)
    if use_date_widget.value:
        filter_list &= (df['month'] == month_widget.value)
    temp_df = df[filter_list]

    with fig_hist.batch_update():
        fig_hist.data[0].x = temp_df['arr_delay']
        fig_hist.data[1].x = temp_df['dep_delay']

update_histogram("unimportant text") # useful for triggering first response

for widget in [airline_widget, origin_widget, month_widget, use_date_widget]:
    widget.observe(update_histogram, names="value")

widget_box_1 = wg.VBox([use_date_widget, month_widget])
widget_box_2 = wg.VBox([airline_widget, origin_widget])
widget_box_main = wg.HBox([widget_box_1, widget_box_2])

wg.VBox([widget_box_main, fig_hist])


### Query tables

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_usa_states.csv')

fig = go.FigureWidget(
    data=[
        go.Table(
            header={"values": list(df.columns),
                    "fill_color": 'paleturquoise',
                    "align": 'left'},
            cells={"values": [df.Rank, df.State, df.Postal, df.Population],
                   "fill_color": 'lavender',
                   "align": 'left'}
        )
    ],
    layout=go.Layout(margin=dict(l=40,r=40,t=40,b=40))
)

label_widget = wg.HTML()

def handle_submit(sender):
    query = sender.value
    try:
        df_reduced = df if query == "" else df.query(query)
        label_widget.value = ""
    except:
        label_widget.value = f"""{repr(query)} is invalid query. Examples: 
        <code><b>State=='Alabama'</b></code>, or <code><b>Postal<'MM' and Population<1000000</b></code>"""
        df_reduced = pd.DataFrame(columns=df.columns)
    fig.data[0].cells.values = [df_reduced.Rank, df_reduced.State, df_reduced.Postal, df_reduced.Population]
        
text_widget = wg.Text(
    value='',
    placeholder='Type something, then press Enter!',
    description='Query:',
    layout=wg.Layout(width="75%")
)
        
text_widget.on_submit(handle_submit)

display(wg.VBox([text_widget, label_widget, fig]))


### Tabs

In [7]:
df0 = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_usa_states.csv')

fig_table = go.FigureWidget(
    data=[
        go.Table(
            header={"values": list(df0.columns),
                    "fill_color": 'paleturquoise',
                    "align": 'left'},
            cells={"values": [df0[col] for col in df0.columns],
                   "fill_color": 'lavender',
                   "align": 'left'}
        )
    ],
    layout=go.Layout(margin=dict(l=40,r=40,t=40,b=40))
)

label_widget = wg.HTML()

def handle_submit(sender):
    query = sender.value
    try:
        df0_reduced = df0 if query == "" else df0.query(query)
        label_widget.value = ""
    except:
        label_widget.value = f"""{repr(query)} is invalid query. Examples: 
        <code><b>State=='Alabama'</b></code>, or <code><b>Postal<'MM' and Population<1000000</b></code>"""
        df0_reduced = pd.DataFrame(columns=df0.columns)
    fig_table.data[0].cells.values = [df0_reduced[col] for col in df0.columns]
        
text_widget = wg.Text(
    value='',
    placeholder='Type something, then press Enter!',
    description='Query:',
    layout=wg.Layout(width="75%")
)

text_widget.on_submit(handle_submit)
        

    
df = pd.read_csv('https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)

month_widget = wg.IntSlider(min=1, max=12, step=1, value=1, description='Month:', continuous_update=False)
use_date_widget = wg.Checkbox(value=True, description='Date: ')
airline_widget = wg.Dropdown(description='Airline:   ', options=df['carrier'].unique(), value='DL')
origin_widget = wg.Dropdown(description='Origin Airport:', options=df['origin'].unique(), value='LGA')

trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
data2 = [trace1, trace2]

layout2 = go.Layout(title='NYC FlightDatabase', 
                    barmode='overlay', 
                    xaxis={"title":"Delay in Minutes"}, 
                    yaxis={"title":"Number of Delays"})

fig_hist = go.FigureWidget(data=data2, layout=layout2)


def update_histogram(change):

    filter_list = (df["carrier"] == airline_widget.value) & (df["origin"] == origin_widget.value)
    if use_date_widget.value:
        filter_list &= (df['month'] == month_widget.value)
    temp_df = df[filter_list]

    with fig_hist.batch_update():
        fig_hist.data[0].x = temp_df['arr_delay']
        fig_hist.data[1].x = temp_df['dep_delay']
        
update_histogram("unimportant text") # useful for triggering first response



for widget in [airline_widget, origin_widget, month_widget, use_date_widget]:
    widget.observe(update_histogram, names="value")

box_layout = wg.Layout(
    border='solid 1px gray',
    margin='0px 10px 10px 0px',
    padding='5px 5px 5px 5px')
    
children = [
    wg.VBox([text_widget, label_widget, fig_table]), 
    wg.VBox([
        wg.HBox([
            wg.VBox([use_date_widget, month_widget], layout=box_layout), 
            wg.VBox([airline_widget, origin_widget], layout=box_layout)
        ]), 
        fig_hist
    ])
]

tab = wg.Tab(children = children)
tab.set_title(0, 'box #1')
tab.set_title(1, 'box #2')


In [8]:
display(tab)
